<a href="https://colab.research.google.com/github/hamin32/boostcourse_assignment/blob/main/%ED%95%99%EC%88%A0%EC%A0%9C_9%ED%98%B8%EC%84%A0%EA%B3%BC%20%EC%88%98%EC%9D%B8%EB%B6%84%EB%8B%B9%EC%84%A0%EC%9D%84%20%ED%99%9C%EC%9A%A9%ED%95%9C%20%EC%99%B8%EA%B3%BD%EB%AC%BC%EB%A5%98%EC%9D%98%20%EC%84%9C%EC%9A%B8%20%EB%8F%84%EC%8B%AC%20%EC%9A%B4%EC%86%A1%20%EC%B2%B4%EA%B3%84%20%EC%84%A4%EA%B3%84%EC%95%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 43.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pulp import *

# 데이터 불러오기
df = pd.read_csv('/content/그만하자.csv', encoding='cp949')  # long_distances.csv 같은 형식

districts = df['자치구'].unique()
stations = df['역사명'].unique()

distance_dict = df.set_index(['자치구', '역사명'])['거리_km'].to_dict()
demand_dict = df.set_index('자치구')['수요량'].to_dict()

# 모델 선언
model = LpProblem("Split_Assignment_Model", LpMinimize)

# 연속변수로 선언: x[(구, 역)] = 이 역에서 구 수요를 얼마나 처리하나
x = LpVariable.dicts("x", [(d, s) for d in districts for s in stations], lowBound=0, cat='Continuous')

# 목적함수: 배송비 최소화
model += lpSum(distance_dict[d, s] * x[(d, s)] * 0.15 for d in districts for s in stations)

# 제약 1: 각 구는 전체 수요량만큼 배분되어야 한다
for d in districts:
    model += lpSum(x[(d, s)] for s in stations) == demand_dict[d]

# 제약 2: 각 역은 3000건을 넘지 못함
for s in stations:
    model += lpSum(x[(d, s)] for d in districts) <= 3000

# 최적화
model.solve()

# 결과 정리
assignments = [(d, s, round(x[(d, s)].varValue)) for d in districts for s in stations if x[(d, s)].varValue > 0]
df_result = pd.DataFrame(assignments, columns=['자치구', '역사명', '할당_물량'])

# 역별 총 물량
station_total = df_result.groupby('역사명')['할당_물량'].sum().reset_index()
station_total.columns = ['역사명', '총_처리_물량']

print(df_result.head())
print("\n역별 총 처리 물량:")
print(station_total.sort_values(by='총_처리_물량', ascending=False))



   자치구          역사명  할당_물량
0  강남구         삼성중앙   2569
1  강남구          선정릉   3000
2  강남구           선릉    299
3  강동구         둔촌오륜   2017
4  강동구  올림픽공원(한국체대)   3000

역별 총 처리 물량:
            역사명  총_처리_물량
0            가양     3000
2          강남구청     3000
3         고속터미널     3000
4         국회의사당     3000
5            노들     3000
33       중앙보훈병원     3000
6           노량진     3000
7            논현     3000
8            당산     3000
12          봉은사     3000
11           등촌     3000
15           샛강     3000
13         삼성중앙     3000
20          선정릉     3000
19          선유도     3000
16          서울숲     3000
17           석촌     3000
25       압구정로데오     3000
24           신사     3000
22          신목동     3000
21         송파나루     3000
35         한성백제     3000
30           염창     3000
31  올림픽공원(한국체대)     3000
32        종합운동장     3000
27         양천향교     3000
26     양재(서초구청)     3000
28           언주     3000
29          여의도     3000
36    흑석(중앙대입구)     3000
34           증미     3000
9       동작(현충원)     2117


In [ ]:
# 역 기준으로 커버하는 자치구별 정보 출력
grouped = df_result.groupby('역사명')

for station, group in grouped:
    print(f"\n🚉 {station} 커버 자치구:")
    for _, row in group.iterrows():
        print(f" - {row['자치구']}: {row['할당_물량']}건")



🚉 가양 커버 자치구:
 - 강서구: 3000건

🚉 강남 커버 자치구:
 - 서초구: 1304건

🚉 강남구청 커버 자치구:
 - 동대문구: 3000건

🚉 고속터미널 커버 자치구:
 - 용산구: 1633건
 - 중구: 1367건

🚉 국회의사당 커버 자치구:
 - 구로구: 425건
 - 서대문구: 2575건

🚉 노들 커버 자치구:
 - 동작구: 3000건

🚉 노량진 커버 자치구:
 - 관악구: 1964건
 - 동작구: 1036건

🚉 논현 커버 자치구:
 - 강북구: 1467건
 - 성북구: 1533건

🚉 당산 커버 자치구:
 - 영등포구: 3000건

🚉 동작(현충원) 커버 자치구:
 - 관악구: 2117건

🚉 둔촌오륜 커버 자치구:
 - 강동구: 2017건

🚉 등촌 커버 자치구:
 - 강서구: 1707건
 - 양천구: 1293건

🚉 봉은사 커버 자치구:
 - 노원구: 3000건

🚉 삼성중앙 커버 자치구:
 - 강남구: 2569건
 - 노원구: 431건

🚉 삼전 커버 자치구:
 - 광진구: 633건

🚉 샛강 커버 자치구:
 - 구로구: 867건
 - 금천구: 2133건

🚉 서울숲 커버 자치구:
 - 노원구: 596건
 - 도봉구: 2404건

🚉 석촌 커버 자치구:
 - 송파구: 3000건

🚉 선릉 커버 자치구:
 - 강남구: 299건

🚉 선유도 커버 자치구:
 - 마포구: 884건
 - 서대문구: 745건
 - 양천구: 233건
 - 영등포구: 1138건

🚉 선정릉 커버 자치구:
 - 강남구: 3000건

🚉 송파나루 커버 자치구:
 - 송파구: 3000건

🚉 신목동 커버 자치구:
 - 마포구: 3000건

🚉 신반포 커버 자치구:
 - 용산구: 630건

🚉 신사 커버 자치구:
 - 성북구: 3000건

🚉 압구정로데오 커버 자치구:
 - 도봉구: 785건
 - 동대문구: 736건
 - 성동구: 1479건

🚉 양재(서초구청) 커버 자치구:
 - 서초구: 3000건

🚉 양천향교 커버 자치구:
 - 은평구: 3000건

🚉 

In [ ]:
pip install folium


In [ ]:
import folium
from folium import PolyLine

# 지도 중심 설정
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 좌표 사전 로드
gu_coords = pd.read_csv("/content/___________.csv")  # 자치구, 위도, 경도
station_coords = pd.read_csv("/content/서울시 역사마스터 정보.csv",encoding='cp949')  # 역사명, 위도, 경도

gu_coords_dict = gu_coords.set_index('자치구')[['위도', '경도']].T.to_dict()
station_coords_dict = station_coords.set_index('역사명')[['위도', '경도']].T.to_dict()

# ✅ 사용된 역사만 추출
used_stations = df_result['역사명'].unique()

# 최적화 결과 기반 선 그리기
for _, row in df_result.iterrows():
    gu = row['자치구']
    st = row['역사명']
    qty = row['할당_물량']

    if gu in gu_coords_dict and st in station_coords_dict:
        gu_lat, gu_lon = gu_coords_dict[gu]['위도'], gu_coords_dict[gu]['경도']
        st_lat, st_lon = station_coords_dict[st]['위도'], station_coords_dict[st]['경도']

        PolyLine([(gu_lat, gu_lon), (st_lat, st_lon)],
                 tooltip=f"{gu} → {st} ({qty}건)",
                 color='blue',
                 weight=2 + qty / 1000).add_to(m)

# 자치구 마커
for gu, coord in gu_coords_dict.items():
    folium.CircleMarker(
        location=[coord['위도'], coord['경도']],
        radius=3,
        color='black',
        fill=True,
        fill_color='black',
        popup=gu
    ).add_to(m)

# ✅ 실제 사용된 역사만 마커 표시
for st in used_stations:
    if st in station_coords_dict:
        coord = station_coords_dict[st]
        folium.Marker(
            location=[coord['위도'], coord['경도']],
            popup=st,
            icon=folium.Icon(color='red', icon='subway', prefix='fa')
        ).add_to(m)

# 저장 또는 보기
m.save("used_station_map.html")
m



<ipython-input-12-f132dea67816>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  station_coords_dict = station_coords.set_index('역사명')[['위도', '경도']].T.to_dict()


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.cluster import KMeans

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

# CSV 불러오기
df = pd.read_csv("/content/물류창고_좌표_면적_최종.csv", encoding='cp949')


# 결측치 제거
df = df.dropna(subset=["위도", "경도"])

# 2. 서울 기준 거리 계산
seoul_center = (37.55, 126.98)
df["서울거리_km"] = df.apply(lambda row: geodesic((row["위도"], row["경도"]), seoul_center).km, axis=1)

# 3. 반경 120km 이내 필터링
df_120 = df[df["서울거리_km"] <= 120].copy()

# 4. 클러스터링 (3개 권역)
coords = df_120[["위도", "경도"]]
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
df_120["클러스터"] = kmeans.fit_predict(coords)

# 5. 각 권역의 가중 중심 계산
centers = []
for i in range(3):
    cluster = df_120[df_120["클러스터"] == i]
    total = cluster["총면적(㎡)"].sum()
    lat = (cluster["위도"] * cluster["총면적(㎡)"]).sum() / total
    lon = (cluster["경도"] * cluster["총면적(㎡)"]).sum() / total
    centers.append({"권역": f"서울권_{i+1}", "위도": lat, "경도": lon})

# 결과 확인
df_result = pd.DataFrame(centers)
print(df_result)

      권역         위도          경도
0  서울권_1  37.571165  126.702794
1  서울권_2  37.146017  127.127466
2  서울권_3  37.275109  127.419383


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.cluster import KMeans
from pulp import *
import folium

# -----------------------------
# 1. 물류창고 클러스터링
# -----------------------------
df = pd.read_csv("/content/물류창고_좌표_면적_최종.csv", encoding='cp949')
df = df.dropna(subset=["위도", "경도"])

seoul_center = (37.55, 126.98)
df["서울거리_km"] = df.apply(lambda row: geodesic((row["위도"], row["경도"]), seoul_center).km, axis=1)
df_120 = df[df["서울거리_km"] <= 120].copy()

coords = df_120[["위도", "경도"]]
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
df_120["클러스터"] = kmeans.fit_predict(coords)

centers = []
for i in range(3):
    cluster = df_120[df_120["클러스터"] == i]
    total = cluster["총면적(㎡)"].sum()
    lat = (cluster["위도"] * cluster["총면적(㎡)"]).sum() / total
    lon = (cluster["경도"] * cluster["총면적(㎡)"]).sum() / total
    centers.append({"권역": f"서울권_{i+1}", "위도": lat, "경도": lon})
df_result = pd.DataFrame(centers)

# -----------------------------
# 2. 물량 최적화 모델
# -----------------------------
df2 = pd.read_csv('/content/그만하자.csv', encoding='cp949')
districts = df2['자치구'].unique()
stations = df2['역사명'].unique()

distance_dict = df2.set_index(['자치구', '역사명'])['거리_km'].to_dict()
demand_dict = df2.set_index('자치구')['수요량'].to_dict()

model = LpProblem("Split_Assignment_Model", LpMinimize)
x = LpVariable.dicts("x", [(d, s) for d in districts for s in stations], lowBound=0, cat='Continuous')

model += lpSum(distance_dict[d, s] * x[(d, s)] * 0.15 for d in districts for s in stations)
for d in districts:
    model += lpSum(x[(d, s)] for s in stations) == demand_dict[d]
for s in stations:
    model += lpSum(x[(d, s)] for d in districts) <= 3000
model += x[("동작구", "흑석(중앙대입구)")] >= 1235

model.solve()

assignments = [(d, s, round(x[(d, s)].varValue)) for d in districts for s in stations if x[(d, s)].varValue > 0]
df_assign = pd.DataFrame(assignments, columns=['자치구', '역사명', '할당_물량'])
station_total = df_assign.groupby('역사명')['할당_물량'].sum().reset_index()
station_total.columns = ['역사명', '총_처리_물량']

# -----------------------------
# 3. 좌표 파일 불러오기 (열 이름 정리 포함)
# -----------------------------
station_coords = pd.read_csv("/content/서울시 역사마스터 정보.csv", encoding='cp949')
station_coords.columns = station_coords.columns.str.strip()

district_coords = pd.read_csv("/content/___________.csv")  # 자치구 좌표
district_coords.columns = district_coords.columns.str.strip()

# -----------------------------
# 4. 지도 시각화 (Folium)
# -----------------------------
import folium

m = folium.Map(location=[37.55, 126.98], zoom_start=10)

import pandas as pd
import folium

# 1. 역사 좌표 데이터 불러오기
station_coords = pd.read_csv("/content/서울시 역사마스터 정보.csv", encoding='cp949')
station_coords.columns = station_coords.columns.str.strip()

# 2. 수인분당선 & 9호선 필터링
line_9 = station_coords[station_coords["호선"] == "9호선"]
line_sb = station_coords[station_coords["호선"] == "분당선, 수인선"]

# ... (물류센터 마커, 자치구 점, 지하철역 마커 등 추가)

from geopy.distance import geodesic

# 두 호선 병합
line_sb = station_coords[
    station_coords["호선"].isin(["분당선", "수인선"])
][["역사명", "위도", "경도"]].copy().dropna()

# 1. 출발점: 경도 기준으로 가장 서쪽 역부터 시작
start_idx = line_sb["경도"].idxmin()
current = line_sb.loc[start_idx]
visited = [start_idx]
path = [ (current["위도"], current["경도"]) ]

# 2. 가장 가까운 역을 하나씩 연결
while len(visited) < len(line_sb):
    remaining = line_sb.drop(index=visited)
    current_coord = (current["위도"], current["경도"])

    # 거리 계산
    remaining["거리"] = remaining.apply(
        lambda row: geodesic((row["위도"], row["경도"]), current_coord).meters, axis=1
    )

    next_idx = remaining["거리"].idxmin()
    current = line_sb.loc[next_idx]
    visited.append(next_idx)
    path.append((current["위도"], current["경도"]))

# 3. PolyLine으로 지도에 추가
folium.PolyLine(
    locations=path,
    color='orange',
    weight=5,
    opacity=0.9,
    tooltip="수인·분당선 (가까운 역 기반)"
).add_to(m)

# ✅ 그 다음에 지도 저장
m.save("최종_물류_지도_노선정리.html")

# 1. 개화역
folium.Marker(
    location=[37.572100, 126.801760],
    popup="개화역",
    icon=folium.Icon(color='green', icon='train', prefix='fa')
).add_to(m)

# 2. 죽전역
folium.Marker(
    location=[37.324670, 127.107180],
    popup="죽전역",
    icon=folium.Icon(color='green', icon='train', prefix='fa')
).add_to(m)


# 1. 클러스터 중심 표시 (그대로)
for _, row in df_result.iterrows():
    folium.Marker(
        location=[row["위도"], row["경도"]],
        popup=row["권역"],
        icon=folium.Icon(color='blue', icon='home')
    ).add_to(m)

# 2. 지하철역 위치 마커 (아이콘으로 강조)
for _, row in station_merged.iterrows():
    folium.Marker(
        location=[row["위도"], row["경도"]],
        popup=f"{row['역사명']}<br>물량: {row['총_처리_물량']}",
        icon=folium.Icon(color='red', icon='train', prefix='fa')  # FontAwesome 사용
    ).add_to(m)

# 3. 자치구 ↔ 역사 연결선 (더 진하고 굵게)
for _, row in df_assign.iterrows():
    d_row = district_coords[district_coords["자치구"] == row["자치구"]]
    s_row = station_coords[station_coords["역사명"] == row["역사명"]]

    if not d_row.empty and not s_row.empty:
        d_lat, d_lon = d_row.iloc[0]["위도"], d_row.iloc[0]["경도"]
        s_lat, s_lon = s_row.iloc[0]["위도"], s_row.iloc[0]["경도"]

        # 선: 더 굵고 진하게
        folium.PolyLine(
            locations=[(d_lat, d_lon), (s_lat, s_lon)],
            color='blue',
            weight=3,
            opacity=0.8
        ).add_to(m)

        # 자치구 지점: 검은 점
        folium.CircleMarker(
            location=(d_lat, d_lon),
            radius=3,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=1
        ).add_to(m)

# 지도 저장 및 보기
m.save("강조된_물류_지도.html")



In [ ]:
from IPython.display import display

# 지도 HTML 객체 출력
display(m)
